In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import numpy as np
import os
from tqdm import tqdm
from pathlib import Path
import multiprocessing
import sys

def create_binary_foreground_image(args):
    image_path, output_image_path = args
    try:
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"이미지를 {image_path}에서 불러올 수 없습니다.", file=sys.stderr)
            return False

        mask = np.zeros(image.shape[:2], np.uint8)

        bgdModel = np.zeros((1, 65), np.float64)
        fgdModel = np.zeros((1, 65), np.float64)

        height, width = image.shape[:2]
        rect = (10, 10, width - 20, height - 20)

        cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

        mask_foreground = np.where((mask == cv2.GC_FGD) | (mask == cv2.GC_PR_FGD), 1, 0).astype('uint8')

        binary_image = (mask_foreground * 255).astype(np.uint8)

        cv2.imwrite(str(output_image_path), binary_image)

        return True
    except Exception as e:
        print(f"이미지 {image_path} 처리 중 오류 발생: {e}", file=sys.stderr)
        return False

def process_images(input_dir, output_dir, start_num=1, end_num=8189, num_workers=None):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    output_dir.mkdir(parents=True, exist_ok=True)

    image_args = []
    for i in range(start_num, end_num + 1):
        image_name = f"image_{i:05d}.jpg"
        image_path = input_dir / image_name

        output_image_name = f"foreground_{i:05d}.png"
        output_image_path = output_dir / output_image_name

        if image_path.exists():
            image_args.append((image_path, output_image_path))
        else:
            print(f"이미지 {image_path}가 존재하지 않습니다. 건너뜁니다.", file=sys.stderr)

    if num_workers is None:
        num_workers = multiprocessing.cpu_count()

    with multiprocessing.Pool(processes=num_workers) as pool:
        results = list(tqdm(pool.imap_unordered(create_binary_foreground_image, image_args),
                            total=len(image_args), desc="Processing Images"))

    success_count = sum(results)
    failure_count = len(results) - success_count
    print(f"\n처리 완료: 성공 {success_count}, 실패 {failure_count}")

if __name__ == "__main__":
    input_image_dir = '/content/drive/MyDrive/심층신경망개론/jpg/'
    output_foreground_dir = '/content/drive/MyDrive/심층신경망개론/foregrounds/'

    process_images(
        input_dir=input_image_dir,
        output_dir=output_foreground_dir,
        start_num=1,
        end_num=8189,
        num_workers=None
    )

Processing Images: 100%|██████████| 8189/8189 [1:11:35<00:00,  1.91it/s]


처리 완료: 성공 8189, 실패 0
